# Очистка, геокодирование и Feature Engineering

В этом ноутбуке мы последовательно:

1. **Загрузка и базовая очистка**  
   - Считываем исходный файл  
   - Удаляем нерелевантные признаки  
   - Фильтруем записи с пустыми улицами и номерами дома  
   - Инициализируем столбцы `lat`, `lon`, `geo_quality`

2. **Геокодирование адресов (дом → улица)**  
   1. Первый проход: точечный геокодинг на уровне дома (`geo_quality = 1`)  
   2. Второй проход: центр улицы + кеширование (`geo_quality = 0`)

3. **Заполнение районов и ближайшей станции метро**  
   - Spatial-join точек (`lat`,`lon`) с полигонами районов → `district`  
   - KD-Tree ближайших станций метро → `underground`

4. **Очистка и сохранение результата**  
   - Удаляем остаточные пропуски по координатам и районам  
   - Сохраняем очищенный датасет в CSV

5. **Расчёт дополнительных фичей**  
   - Расстояние до **центра города**  
   - Расстояние до **ближайшего метро** 
   - Наличие **парка** в радиусе 1 км  
   - Количество объектов соц. инфраструктуры (школы, поликлиники, магазины, аптеки) в радиусе 1 км  

---

**Цель:** получить чистый, географически обогащённый и готовый к обучению ML-датасет для предсказания справедливой цены аренды.  

# 1. Импорт библиотек
___

In [1]:
!pip install --quiet osmnx geopandas shapely numpy pandas scipy geopy tqdm joblib fiona osmium

In [2]:
from __future__ import annotations

import math
import re
import warnings
from pathlib import Path
from typing import Any, Dict, Iterable, List, Tuple
import json
import os

import joblib
import numpy as np
import osmium
import pandas as pd
import pyproj
import requests
import shapely.wkt
import geopandas as gpd

from tqdm.auto import tqdm
from scipy.spatial import cKDTree
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Настройки
warnings.filterwarnings("ignore", category=UserWarning)
tqdm.pandas()

## 2. Загрузка и базовая очистка данных

*Что происходит:*  
- Считываем исходный файл `cian_rent_Москва_balanced.xls`.  
- Удаляем нерелевантные колонки (`USELESS_COLS`).  
- Приводим пустые строки к `pd.NA` для ключевых столбцов (`LOC_COLS`).  
- Фильтруем записи без улицы или номера дома.  
- Инициализируем столбцы `lat`, `lon`, `geo_quality`.
___


In [3]:
# Загрузка и базовая очистка датасета
RAW_PATH = "cian_rent_Moscow_balanced.xls"

USELESS_COLS = [
    "author", "author_type", "url", "location",
    "deal_type", "accommodation_type", "commissions",
    "residential_complex",
]
LOC_COLS = ["underground", "house_number", "street", "district"]

data_raw: pd.DataFrame = pd.read_csv(RAW_PATH)

data_useful: pd.DataFrame = (
    data_raw
    .drop(USELESS_COLS, axis=1, errors="ignore")
    .assign(**{c: lambda df, c=c: df[c].replace("", pd.NA) for c in LOC_COLS})
    .dropna(subset=["street", "house_number"])
    .assign(lat=pd.NA, lon=pd.NA, geo_quality=pd.NA)          # geo_quality: 1 точно, 0 улица
    .reset_index(drop=True)
)

print(f"После очистки: {len(data_useful):,} объявлений")

После очистки: 16,114 объявлений


## 3. Загрузка геоданных: районы и метро

*Что происходит:*  
- Загружаем полигоны районов Москвы из GeoJSON → `districts`.  
- Переводим CRS в `EPSG:4326` и переименовываем колонку района в `district_name`.  
- Скачиваем или читаем CSV со станциями метро, строим `cKDTree` для быстрого поиска ближайшей станции.

___


In [4]:
def load_districts_and_metro(
    districts_path: str = "mo.geojson",
    metro_csv_path: str = "moscow_metro_stations.csv"
) -> tuple[gpd.GeoDataFrame, pd.DataFrame, cKDTree]:
    # Загрузка и обработка районов
    def rename_columns(c: str) -> str:
        cl = c.lower()
        if cl.startswith("name") and c != "geometry":
            return "district_name"
        elif cl.startswith("adm") or "округ" in cl or "area" in cl:
            return "adm_area"
        return c

    districts = (
        gpd.read_file(districts_path)
        .to_crs("EPSG:4326")
        .rename(columns=rename_columns)
    )

    # Только нужные колонки
    district_cols = [c for c in ["district_name", "adm_area", "geometry"] if c in districts.columns]
    districts = districts[district_cols]

    # Загрузка и обработка метро
    metro_csv = Path(metro_csv_path)
    if not metro_csv.exists():
        print("Скачиваем станции метро из API HeadHunter...")
        response = requests.get("https://api.hh.ru/metro/1")
        data = response.json()
        stations = []
        for line in data['lines']:
            for station in line['stations']:
                stations.append({
                    'name': station['name'],
                    'lat': station['lat'],
                    'lon': station['lng']
                })
        metro_df = pd.DataFrame(stations)
        metro_df.to_csv(metro_csv, index=False, encoding="utf-8-sig")
        print(f"Станции метро сохранены в {metro_csv_path}")
    else:
        metro_df = pd.read_csv(metro_csv)

    # Убираем все лишнее
    metro_df = metro_df[["name", "lat", "lon"]]

    # KD-дерево
    metro_tree = cKDTree(metro_df[["lat", "lon"]].to_numpy())

    return districts, metro_df, metro_tree

In [5]:
districts, metro_df, metro_tree = load_districts_and_metro()

## 4. Нормализация адресов

*Что происходит:*  
- Убираем префиксы улиц (`ул.`, `проспект`, `г.`, и т.п.) при помощи регулярного выражения.  
- Удаляем точки и запятые, приводим к нижнему регистру.  
- Формируем строку вида `Россия, Москва, <улица> <дом>`, готовую для геокодера.

___


In [8]:
# Нормализация адреса
_addr_head = re.compile(r"^\s*(ул\.?|улица|проспект|пр[-‐-]?(кт|т)|г\.?|город)\s*", flags=re.I)

def normalize_address(row: pd.Series) -> str:
    street = _addr_head.sub("", str(row.street).strip().lower())
    street = re.sub(r"[.,]", " ", street).strip()
    return f"Россия, Москва, {street} {row.house_number}"

## 5. Геокодирование адресов (двухэтапное)

*Что происходит:*  
1. **Первый проход (house-level):**  
   - Геокодим каждый адрес на уровне дома через Nominatim.  
   - Помечаем `geo_quality = 1`.  
2. **Второй проход (street-level):**  
   - Собираем уникальные названия улиц без координат, геокодим центры улиц, кешируем.  
   - Подставляем из кеша в оставшиеся пропуски, `geo_quality = 0`.  
3. Удаляем все записи, у которых так и не появились координаты.

___

In [9]:
# Геокодер и двухэтапное заполнение координат
geocoder = Nominatim(user_agent="rental-geocoder", timeout=10)
rate_limited = RateLimiter(geocoder.geocode, min_delay_seconds=1.1, swallow_exceptions=False)

GOOD_TYPES: set[str] = {"house", "building", "apartments", "residential", "yes"}
STREET_TYPES: set[str] = {"residential", "road", "street", "tertiary",
                          "secondary", "primary", "motorway", "service"}

def geocode_rows(rows: Iterable[Tuple[int, pd.Series]],
                 street_cache: Dict[str, Tuple[float, float]] | None = None) -> None:
    """Заполняет lat/lon и geo_quality в исходном data_useful по индексам rows."""
    for idx, row in rows:
        try:
            loc = rate_limited(normalize_address(row))
        except (GeocoderTimedOut, GeocoderServiceError):
            continue

        if loc and loc.raw.get("type") in GOOD_TYPES:
            data_useful.at[idx, "lat"] = loc.latitude
            data_useful.at[idx, "lon"] = loc.longitude
            data_useful.at[idx, "geo_quality"] = 1
        elif street_cache is not None:
            s_key = str(row.street).lower().strip()
            coords = street_cache.get(s_key)
            if coords:
                data_useful.at[idx, "lat"], data_useful.at[idx, "lon"] = coords
                data_useful.at[idx, "geo_quality"] = 0

# — ПЕРВЫЙ ПРОХОД —
mask_blank = data_useful["lat"].isna() & data_useful["lon"].isna()
print("House-level pass…")
geocode_rows(tqdm(data_useful.loc[mask_blank].iterrows(),
                  total=mask_blank.sum(), desc="house"))

# — ВТОРОЙ ПРОХОД (центр улицы) —
mask_blank = data_useful["lat"].isna() & data_useful["lon"].isna()
if mask_blank.any():
    streets = (
        data_useful.loc[mask_blank, "street"]
        .str.lower().str.strip().dropna().unique()
    )
    street_cache: Dict[str, Tuple[float, float]] = {}
    print("Building street-centroid cache…")
    for st in tqdm(streets, desc="street"):
        try:
            loc = rate_limited(f"Россия, Москва, {st}")
        except (GeocoderTimedOut, GeocoderServiceError):
            continue
        if loc and loc.raw.get("type") in STREET_TYPES:
            street_cache[st] = (loc.latitude, loc.longitude)

    if street_cache:
        print("Street-level pass…")
        geocode_rows(tqdm(data_useful.loc[mask_blank].iterrows(),
                          total=mask_blank.sum(), desc="street"),
                     street_cache=street_cache)

# Финальная чистка
before = len(data_useful)
data_useful.dropna(subset=["lat", "lon"], inplace=True)
after = len(data_useful)
print(f"Удалено строк без координат: {before - after} (осталось {after})")

House-level pass…


house:   0%|          | 0/16114 [00:00<?, ?it/s]

Building street-centroid cache…


street:   0%|          | 0/489 [00:00<?, ?it/s]

Street-level pass…


street:   0%|          | 0/2926 [00:00<?, ?it/s]

Удалено строк без координат: 980 (осталось 15134)


## 6. Заполнение районов и ближайшей станции метро

*Что происходит:*  
- Создаём GeoDataFrame точек (`lat`, `lon`) и делаем spatial-join с полигонами районов → заполняем `district`.  
- Для оставшихся NaN в `underground` используем `metro_tree.query(k=1)` → записываем ближайшую станцию.
- Контролируем, чтобы в обоих столбцах не осталось пропусков.

___

In [10]:
def enrich_with_districts_and_metro(
    data_useful: pd.DataFrame,
    districts: gpd.GeoDataFrame,
    metro_df: pd.DataFrame,
    metro_tree: cKDTree
) -> pd.DataFrame:
    """
    Обогащает датафрейм data_useful районом (по координатам) и ближайшим метро (по KD-дереву).
    """

    data_useful = data_useful.loc[:, ~data_useful.columns.duplicated()]

    # Собираем GeoDataFrame точек с тем же индексом
    points = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy(data_useful.lon, data_useful.lat),
        crs="EPSG:4326",
        index=data_useful.index
    )

    # Spatial join с полигонами районов
    joined = gpd.sjoin(
        points,
        districts[["district_name", "geometry"]],
        how="left",
        predicate="within"
    )

    if "index_left" in joined.columns:
        joined = joined.set_index("index_left", drop=True)

    district_cols = [col for col in joined.columns if col == "district_name"]

    if len(district_cols) > 1:
        district_series = joined[district_cols].iloc[:, 0]
    else:
        district_series = joined["district_name"]

    # Группируем по индексу и берём первое значение (на случай дублей геометрий)
    district_map = district_series.groupby(level=0).first()

    # Заполняем пропуски в столбце district
    data_useful["district"] = data_useful["district"].fillna(district_map)

    # Теперь подземка через KD-tree
    mask_und = data_useful["underground"].isna()
    if mask_und.any():
        coords = data_useful.loc[mask_und, ["lat", "lon"]].to_numpy()
        _, idx = metro_tree.query(coords, k=1)
        data_useful.loc[mask_und, "underground"] = metro_df["name"].iloc[idx].to_numpy()

    print("NaN district     :", data_useful["district"].isna().sum())
    print("NaN underground :", data_useful["underground"].isna().sum())

    return data_useful

data_useful = enrich_with_districts_and_metro(data_useful, districts, metro_df, metro_tree)

NaN district     : 32
NaN underground : 0


## 6. Очистка и сохранение результата

*Что происходит:*  
- Удаляем остаточные записи без значения `district`.  
- Сохраняем итоговый обогащённый датасет в `cian_rent_Moscow_cleaned.csv` (UTF-8 BOM).

---

In [11]:
# Cell 2: отброс пропусков по district, сохранение
data_useful.dropna(subset=["district"], inplace=True)

output_path = "cian_rent_Moscow_cleaned.csv"
data_useful.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Очищенный датасет сохранён: {len(data_useful):,} строк → {output_path}")

Очищенный датасет сохранён: 15,102 строк → cian_rent_Moscow_cleaned.csv


## 7. Загрузка данных и функция Haversine

*Что происходит:*  
- Загружаем очищенный датасет `cian_rent_Moscow_cleaned.csv`.  
- Определяем функцию `haversine()` для расчёта расстояния между двумя географическими точками в километрах.  
- Эта функция понадобится для расчёта расстояний до центра Москвы и ближайшего метро.

---

In [12]:
df = pd.read_csv("cian_rent_Moscow_cleaned.csv")

def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    R = 6371.0
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    Δφ = math.radians(lat2 - lat1)
    Δλ = math.radians(lon2 - lon1)
    a = math.sin(Δφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(Δλ/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

## 8. Расчёт расстояния до центра города

*Что происходит:*  
- Добавляем колонку `dist_to_center_km`, в которой указано расстояние от квартиры до центра Москвы.  
- За центр принимаем координаты Красной площади: `(55.752023, 37.617499)`.  
- Используем функцию `haversine()` построчно для расчёта.

---

In [13]:
def add_distance_to_center(
    df: pd.DataFrame,
    center_coords: Tuple[float, float] = (55.752023, 37.617499),
    lat_col: str = "lat",
    lon_col: str = "lon",
    new_col: str = "dist_to_center_km"
) -> pd.DataFrame:
    center_lat, center_lon = center_coords
    df[new_col] = df.apply(
        lambda row: haversine(row[lat_col], row[lon_col], center_lat, center_lon),
        axis=1
    )
    return df

df = add_distance_to_center(df)

## 9. Нормализация и KD-fallback для станции метро

*Что происходит:*  
- Загружаем справочник станций метро (`moscow_metro_stations.csv`).  
- Нормализуем названия станций:  
  - заменяем `ё` на `е`,  
  - удаляем слова "станция", "МЦК", "МЦД" и текст в скобках,  
  - устраняем лишние пробелы и приводим текст к нижнему регистру.  
- Маппим полученные строки на официальные названия станций.  
- Для строк, где не удалось найти метро, используем KD-дерево:  
  - определяем ближайшую станцию метро по координатам.

---

In [14]:
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.replace("ё", "е")
    s = re.sub(r"[—–]+", "-", s)
    s = re.sub(r"\s*\(.*?\)", "", s, flags=re.I)
    s = re.sub(r"\b(станция|мцк|мцд)\b", "", s, flags=re.I)
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip().lower()

def build_station_mapper(metro_df: pd.DataFrame) -> dict:
    return {
        normalize_text(name): name
        for name in metro_df["name"].dropna().unique()
    }

metro_df = pd.read_csv("moscow_metro_stations.csv")
mapper   = build_station_mapper(metro_df)

df["underground"] = (
    df["underground"]
      .fillna("")            
      .astype(str)
      .apply(normalize_text)
      .map(mapper)
)

mask = df["underground"].isna()
if mask.any():
    coords = df.loc[mask, ["lat", "lon"]].to_numpy()
    _, idx = metro_tree.query(coords, k=1)
    df.loc[mask, "underground"] = metro_df["name"].iloc[idx].to_numpy()

print("Осталось NaN в underground:", df["underground"].isna().sum())

Осталось NaN в underground: 0


## 10. Расчёт расстояния до метро

*Что происходит:*  
- Добавляем колонку `dist_to_metro_km` — расстояние от квартиры до указанной станции метро.  
- Используем справочник координат станций метро.  
- Расчёт выполняется функцией `haversine()`.

---

In [15]:
def add_distance_to_metro(
    df: pd.DataFrame,
    metro_df: pd.DataFrame,
    lat_col: str = "lat",
    lon_col: str = "lon",
    station_col: str = "underground",
    new_col: str = "dist_to_metro_km"
) -> pd.DataFrame:
    station_coords: Dict[str, Tuple[float, float]] = {
        row["name"]: (row["lat"], row["lon"])
        for _, row in metro_df.iterrows()
    }
    def compute_dist(row):
        coords = station_coords.get(row[station_col])
        return float("nan") if coords is None else haversine(
            row[lat_col], row[lon_col], coords[0], coords[1]
        )
    df[new_col] = df.apply(compute_dist, axis=1)
    return df

df = add_distance_to_metro(df, metro_df)
print("Осталось NaN в dist_to_metro_km:", df["dist_to_metro_km"].isna().sum())

Осталось NaN в dist_to_metro_km: 0


## 11. Сохранение новой версии датасета

*Что происходит:*  
- Сохраняем новый датафрейм в файл `cian_rent_Moscow_with_dist.csv`.  
- Включены новые полезные признаки: расстояние до центра и растояние до ближайшего метро.  

---

In [23]:
output_path = "cian_rent_Moscow_with_dist.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Новый датасет сохранён: {len(df):,} строк → {output_path}")

Новый датасет сохранён: 15,102 строк → cian_rent_Moscow_with_dist.csv


## 12. Параметры и категории POI

*Что происходит:*    
- Устанавливаем пути к входным и выходным файлам.  
- Задаем категории объектов интереса (POI):  
  - зеленые зоны (`GREEN_LEISURE`),  
  - исключения для неучебных "школ" (`SCHOOL_EXCLUDE`),  
  - ключевые слова для аптек и продуктовых магазинов.

---

In [18]:
PBF_FILE  = "moscow.pbf"
INPUT_CSV = "cian_rent_Moscow_with_dist.csv"
OUTPUT_CSV = "cian_rent_Moscow_with_poi.csv"

GREEN_LEISURE = {"park", "garden", "square", "recreation_ground"}
SCHOOL_EXCLUDE = {
    "грум", "маникюр", "танц", "автошкол", "кулинар",
    "массаж", "церков", "приход", "духовн", "sunday school",
}
PHARMA_KEYS = {
    "апрель", "ригла", "планета здоровья", "имплозия",
    "эркафарм", "мелодия здоровья", "36,6", "неофарм",
    "ирис", "фармленд", "вита",
}
GROCERY_KEYS = {
    "пятероч", "перекресток", "чижик", "магнит",
    "красное & белое", "бристоль", "лента",
    "светофор", "вкусвилл", "auchan",
    "азбука вкуса", "globus gourmet", "bella vita", "dean & deluca",
}
transform = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform
wkt_factory = osmium.geom.WKTFactory()

## 13. Предикаты для фильтрации объектов

*Что происходит:*  
- Определяем предикаты для фильтрации объектов OSM по тегам:  
  - `is_green`: парк, сквер и т.п.  
  - `is_school`: образовательные учреждения за вычетом проф. курсов.  
  - `is_pharma` и `is_grocery`: проверка на наличие ключевых слов.  
  - `is_medical`: поликлиники и больницы.

---


In [19]:
def norm(t: str) -> str: return t.lower().replace("ё", "е") if t else ""

def is_green(tags: Dict[str, str]) -> bool:
    return tags.get("leisure") in GREEN_LEISURE

def is_school(tags: Dict[str, str]) -> bool:
    if tags.get("amenity") != "school" and tags.get("building") != "school":
        return False
    return not any(w in norm(tags.get("name", "")) for w in SCHOOL_EXCLUDE)

def has_kw(tags: Dict[str, str], kws: set[str]) -> bool:
    joined = " ".join(tags.get(k, "") for k in ("name", "brand", "operator"))
    return any(k in norm(joined) for k in kws)

def is_pharma(tags: Dict[str, str]) -> bool:
    return tags.get("amenity") == "pharmacy" and has_kw(tags, PHARMA_KEYS)

def is_grocery(tags: Dict[str, str]) -> bool:
    if tags.get("shop") not in {"supermarket", "convenience", "mall", "department_store"}:
        return False
    return has_kw(tags, GROCERY_KEYS)

def is_medical(tags: Dict[str, str]) -> bool:
    return tags.get("amenity") in {"clinic", "hospital"} or tags.get("healthcare") in {"clinic", "hospital"}

## 14. OSM Handler и утилиты

*Что происходит:*  
- Создаем обработчик `POI` для сбора POI из PBF-файла:  
  - обрабатываем `node`, `way`, `area` из OSM.  
  - сохраняем координаты объектов по категориям.  
- Используем `centroid` для вычисления центра многоугольников.  
- Добавляем функцию `dedup` для удаления близкорасположенных дубликатов.

---


In [20]:
def centroid(entity: Any) -> Tuple[float, float]:
    try:
        if isinstance(entity, osmium.osm.Way) and entity.is_closed():
            geom = shapely.wkt.loads(wkt_factory.create_multipolygon(entity))
        elif isinstance(entity, osmium.osm.Area):
            geom = shapely.wkt.loads(wkt_factory.create_multipolygon(entity))
        else:
            return entity.location.lon, entity.location.lat
        c = geom.centroid
        return c.x, c.y
    except Exception:
        return None, None

class POI(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.parks, self.schools, self.pharm, self.gro, self.med = ([] for _ in range(5))

    def _add(self, lon: float, lat: float, tags: Dict[str, str]) -> None:
        x, y = transform(lon, lat)
        if is_green(tags):
            self.parks.append((x, y))
        elif is_school(tags):
            self.schools.append((x, y))
        elif is_pharma(tags):
            self.pharm.append((x, y))
        elif is_grocery(tags):
            self.gro.append((x, y))
        elif is_medical(tags):
            self.med.append((x, y))

    def node(self, n): self._add(n.location.lon, n.location.lat, dict(n.tags))
    def way(self, w):
        if w.is_closed():
            lon, lat = centroid(w)
            if lon is not None: self._add(lon, lat, dict(w.tags))
    def area(self, a):
        lon, lat = centroid(a)
        if lon is not None: self._add(lon, lat, dict(a.tags))

def dedup(coords: List[Tuple[float, float]], r: float = 100.) -> np.ndarray:
    arr = np.asarray(coords)
    if not len(arr): return arr.reshape(0, 2)
    tree, keep = cKDTree(arr), np.ones(len(arr), bool)
    for i, p in enumerate(arr):
        if keep[i]:
            nbrs = tree.query_ball_point(p, r); nbrs.remove(i); keep[nbrs] = False
    return arr[keep]

## 15. Парсинг PBF и агрегация школ

*Что происходит:*  
- Применяем обработчик POI к файлу `moscow.pbf`.  
- Получаем списки координат объектов.  
- Удаляем дублирующиеся школы, расположенные ближе 100 метров друг к другу.

---


In [21]:
h = POI(); h.apply_file(PBF_FILE, locations=True, idx="flex_mem")
schools = dedup(h.schools)

## 16. Обработка квартир и подсчет POI в радиусе

*Что происходит:*  
- Загружаем датасет с квартирами (`INPUT_CSV`).  
- Преобразуем координаты квартир в метры (проекция EPSG:3857).  
- Для каждой категории POI считаем количество объектов в радиусе 1 км:  
  - `parks_1km`,  
  - `groceries_top_1km`,  
  - `schools_1km`,  
  - `pharmacies_top_1km`,  
  - `clinics_1km`.

---


In [22]:
df = pd.read_csv(INPUT_CSV)
flat_xy = np.vstack([transform(lon, lat) for lon, lat in zip(df.lon, df.lat)])

def add(df: pd.DataFrame, name: str, pts: np.ndarray, r: float) -> None:
    if not len(pts): df[name] = 0; return
    hits = cKDTree(pts).query_ball_point(flat_xy, r)
    df[name] = [len(h) for h in hits]

add(df, "parks_1km", np.asarray(h.parks), 1000)
add(df, "groceries_top_1km", np.asarray(h.gro), 1000)
add(df, "schools_1km", schools, 1000)
add(df, "pharmacies_top_1km", np.asarray(h.pharm), 1000)
add(df, "clinics_1km", np.asarray(h.med), 1000)

In [24]:
output_path = "cian_rent_Moscow_with_poi.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Финальный датасет сохранён: {len(df):,} строк → {output_path}")

Финальный датасет сохранён: 15,102 строк → cian_rent_Moscow_with_poi.csv


## 17. Сохранение артефактов для инференса

*Что происходит:*  
- Сохраняем все необходимые артефакты, чтобы на этапе инференса не выполнять дорогие операции повторно.  
- Это позволяет быстро обогатить новые данные теми же признаками, что использовались при обучении модели.  
- Включает следующие объекты:

  - Геоданные:
    - `districts_processed.geojson` — полигоны районов;
    - `station_mapper.json` — нормализованные названия станций метро;
    - `street_cache.json` — кэш координат для fallback-геокодинга улиц.
  
  - Метро:
    - `metro_tree.pkl` — KD-дерево для поиска ближайшего метро.
  
  - POI:
    - `.npy` — координаты объектов (парки, школы и т.д.);
    - `.pkl` — KD-деревья для быстрого поиска POI в радиусе.

*Зачем нужно:*  
Эти артефакты позволяют:
- моментально определять район по координатам (без spatial join в рантайме),
- определять ближайшее метро и расстояние до него,
- считать количество POI рядом с объектом,
- избавляют от повторного парсинга `.pbf` и геокодирования.

---


In [49]:
os.makedirs("artifacts", exist_ok=True)

In [50]:
districts = districts.loc[:, ~districts.columns.duplicated()]
districts.to_file("artifacts/districts_processed.geojson", encoding="utf-8")

In [51]:
# Сохраняем KD-дерево
joblib.dump(metro_tree, "artifacts/metro_tree.pkl")

['artifacts/metro_tree.pkl']

In [52]:
with open("artifacts/station_mapper.json", "w", encoding="utf-8") as f:
    json.dump(mapper, f, ensure_ascii=False, indent=2)

In [53]:
with open("artifacts/street_cache.json", "w", encoding="utf-8") as f:
    json.dump(street_cache, f, ensure_ascii=False, indent=2)

In [54]:
np.save("artifacts/parks.npy", np.asarray(h.parks))
np.save("artifacts/schools.npy", schools)
np.save("artifacts/pharmacies.npy", np.asarray(h.pharm))
np.save("artifacts/groceries.npy", np.asarray(h.gro))
np.save("artifacts/clinics.npy", np.asarray(h.med))

joblib.dump(cKDTree(np.asarray(h.parks)),      "artifacts/parks_tree.pkl")
joblib.dump(cKDTree(schools),                 "artifacts/schools_tree.pkl")
joblib.dump(cKDTree(np.asarray(h.pharm)),     "artifacts/pharmacies_tree.pkl")
joblib.dump(cKDTree(np.asarray(h.gro)),       "artifacts/groceries_tree.pkl")
joblib.dump(cKDTree(np.asarray(h.med)),       "artifacts/clinics_tree.pkl")

['artifacts/clinics_tree.pkl']